In [112]:
#####RESCAL#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import math
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
import seaborn as sns

#np.random.seed(98537)

In [113]:
####データの発生####
##データの設定
k = 10
d = 5000
m = 12
no = np.arange(d)

In [114]:
##ノードを生成
#セグメントをリンク確率を生成
g = 15
Z = np.random.multinomial(1, np.random.dirichlet(np.repeat(2.0, g), 1).reshape(-1), d)
z_vec = np.dot(Z, np.arange(g))
beta = np.random.beta(0.75, 15.0, g*g).reshape(g, g)
beta = np.triu(beta, k=1) + np.triu(beta).T

#ノードの全パターンを作成
target = np.triu(np.full((d, d), 1), k=1)
D1 = np.triu(np.full((d, d), np.arange(d)).T, k=1)
D2 = np.triu(np.full((d, d), np.arange(d)), k=1)
d1 = D1[target==1]; d2 = D2[target==1]
del target; D1, D2

#ユーザーごとにノードを生成
y = np.random.binomial(1, np.sum(beta[z_vec[d1], ] * Z[d2, ], axis=1), d1.shape[0])
index_y = np.where(y==1)[0]
d_id1 = d1[index_y]; d_id2 = d2[index_y]

#インデックスを作成
target = [i for i in range(d)]
d_list = [i for i in range(d)]
d_list1 = [i for i in range(d)]
d_list2 = [i for i in range(d)]
pt = np.repeat(0, d)
for i in range(d):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int32")
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int32")
    d_list[i] = np.append(d_list1[i], d_list2[i])
    target[i] = np.append(np.repeat(0, d_list1[i].shape[0]), np.repeat(1, d_list2[i].shape[0]))
    pt[i] = target[i].shape[0]
N = np.sum(pt)
hhpt = d_id1.shape[0]

In [115]:
##説明変数の生成
#素性ベクトルを生成
k1 = 4; k2 = 5; k3 = 4
x1 = np.array(np.random.random(hhpt*k1)).reshape((hhpt, k1))
x2 = np.zeros((hhpt, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    x2[:, j] = np.random.binomial(1, prob, hhpt)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hhpt)
x3 = np.delete(x3, np.argmin(np.sum(x3, axis=0)), axis=1)   #冗長な変数の削除
x = np.concatenate((np.repeat(1, hhpt)[:, np.newaxis], x1, x2, x3), axis=1)
x_col = x.shape[1]

#階層モデルの説明変数を生成
k1 = 4; k2 = 5; k3 = 5
u1 = np.array(np.random.random(d*k1)).reshape((d, k1))
u2 = np.zeros((d, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    u2[:, j] = np.random.binomial(1, prob, d)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), d)
u3 = np.delete(u3, np.argmin(np.sum(u3, axis=0)), axis=1)   #冗長な変数の削除
u = np.concatenate((np.repeat(1, d)[:, np.newaxis], u1, u2, u3), axis=1)
u_col = u.shape[1]

In [116]:
####応答変数を生成####
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    ##パラメータを生成
    #階層モデルのパラメータを生成
    alpha = np.random.normal(0, 0.3, k*u_col).reshape(u_col, k)
    Cov = np.diag(np.repeat(0.1, k))
    alphat = alpha.copy(); Covt = Cov.copy()

    #モデルパラメータを生成
    beta0 = np.random.uniform(-2.0, -1.0, m)
    beta1 = np.random.normal(0, 0.3, (x_col-1)*m)
    beta = np.append(beta0, beta1).reshape(x_col, m)
    theta = np.dot(u, alpha) + np.random.multivariate_normal(np.repeat(0, k), Cov, d)
    omega = np.zeros((k, k, m))
    for j in range(m):
        omega[:, :, j] = np.random.normal(0, 0.25, k*k).reshape(k, k)
    betat = beta.copy(); thetat = theta.copy(); omegat = omega.copy()
        
    ##応答変数を生成
    #期待値を設定
    beta_mu = np.dot(x, beta)
    mu = np.zeros((hhpt, m))
    for j in range(m):
        mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

    #正規分布からバイナリー変数を生成
    U = np.random.normal(mu, 1)
    y = np.array(U > 0, dtype="int")
    UT = U.copy()
    
    #break条件
    if (np.min(np.mean(y, axis=0)) > 0.1) & (np.max(np.mean(y, axis=0)) < 0.5):
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [117]:
#生成したデータを確認
print(np.mean(y, axis=0))
print(np.sum(y, axis=0))

[0.14607444 0.45569114 0.11477628 0.28557494 0.17542006 0.16974026
 0.19162797 0.29437067 0.18068835 0.11000755 0.22534097 0.16162191]
[ 86259 269092  67777 168636 103588 100234 113159 173830 106699  64961
 133067  95440]


In [118]:
####マルコフ連鎖モンテカルロ法でRESCALを推定####
##パラメータ推定のための関数
#切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n, k):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    if k==1:
        res = norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)
    else:
        res = norm.ppf(np.random.uniform(0, 1, n*k).reshape(n, k)*(FB-FA)+FA, mu, sigma)
    return res

##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [119]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 250/keep
disp = 10
e = 0.1
L = 3

In [120]:
##事前分布の設定
#階層モデルの事前分布
Deltabar = np.zeros((u_col, k))
ADelta =0.01 * np.diag(np.ones(u_col))
nu = k + 1
V = nu * np.diag(np.ones(k))

#モデルパラメータの事前分布
gamma = np.repeat(0, x_col)
tau = np.diag(np.repeat(100, x_col))
inv_tau = np.linalg.inv(tau)

In [144]:
##パラメータの真値
#階層モデルのパラメータの真値
alpha = alphat.copy()
Cov = Covt.copy()
inv_Cov = np.linalg.inv(Cov)

#モデルパラメータの真値
Sigma = 1
beta = betat.copy()
theta = thetat.copy()
omega = omegat.copy()

#期待値の真値
beta_mu = np.dot(x, beta)
mu = np.zeros((hhpt, m))
for j in range(m):
    mu[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))

In [194]:
#期待値の真値
beta_mu = np.dot(x, beta)
mu1 = np.zeros((hhpt, m))
for j in range(m):
    mu1[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))
    
#期待値の真値
beta_mu = np.dot(x, beta)
mu2 = np.zeros((hhpt, m))
for j in range(m):
    mu2[:, j] = beta_mu[:, j] + np.dot(np.dot(theta, omega[:, :, j])[d_id2, ] * theta[d_id1, ], np.repeat(1, k))

In [195]:
i = 0
print(mu1[i, ])
print(mu2[i, ])

[-2.94046469 -0.06716727 -4.18898142 -0.2113717  -0.92238626 -1.71532004
 -0.79591891 -1.05628947 -2.1673056  -3.05714092 -2.28708951 -1.51416791]
[-2.78918468 -0.02104649 -3.17665406  0.10539715 -0.84665168  0.29399236
 -1.07942544 -0.16414923 -2.64509219 -1.14858531 -2.83569021 -0.81869725]


In [205]:
i = 0
j = 0
beta_mu[i, j] + np.dot(np.dot(theta[1, ], omega[:, :, j].T), theta[0, ])

-1.8167410460932274

In [134]:
##データの設定
#切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

#定数を設定
xxv = np.dot(x.T, x) + inv_tau
inv_xxv = np.linalg.inv(xxv)

In [145]:
####ギブスサンプリングでパラメータをサンプリング####

##切断正規分布から潜在変数をサンプリング
#期待値の設定
beta_mu = np.dot(x, beta)
ARA = np.zeros((hhpt, m))
for j in range(m):
    ARA[:, j] = np.dot(np.dot(theta, omega[:, :, j])[d_id1, ] * theta[d_id2, ], np.repeat(1, k))
mu = beta_mu + ARA
    
#切断領域に応じて潜在変数をサンプリング
U = rtnorm(mu, Sigma, a, b, hhpt, m)

In [140]:
##素性ベクトルの回帰ベクトルをサンプリング
#多変量正規分布のパラメータ
er = U - ARA   #モデル誤差
xy = np.dot(x.T, er)
par_mu = np.dot(inv_xxv, xy)

#回帰ベクトルをサンプリング
for j in range(m):
    beta[:, j] = np.random.multivariate_normal(par_mu[:, j], inv_xxv, 1)
beta_mu = np.dot(x, beta)

In [192]:
#特徴行列をサンプリング
er = U - beta_mu   #モデル誤差

i = 0
j = 0
v = np.dot(theta[id_list[i], ], omega[:, :, j])
np.dot(np.dot(np.linalg.inv(np.dot(v.T, v)), v.T), er[d_list[i], j])

array([ -2.83575552, -12.47161729,   2.16962816,   3.0440136 ,
       -12.39093817,  -3.67353103, -17.34974576,  -6.89629354,
        16.79644459,   1.11644938])

In [193]:
theta[i, ]

array([-0.52218787, -0.30191123,  0.12621053,  0.20733886,  0.86875244,
        0.68551109,  0.24889813,  0.38246148, -0.87963055, -0.06293212])

In [182]:
d_id = np.hstack((d_id1[:, np.newaxis], d_id2[:, np.newaxis]))
id_list = [i for i in range(d)]
for i in range(d):
    id_list[i] = np.sum(np.array(d_id[d_list[i], ]!=i, dtype="int") * d_id[d_list[i], ], axis=1)

In [190]:
d_list[i]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

array([1017, 1019, 1036, 1038, 1044, 1046, 1056, 1058, 1073, 1086, 1104,
       1113, 1129, 1136, 1157, 1187, 1193, 1233, 1280, 1319, 1341, 1343,
       1348, 1361, 1381, 1419, 1439, 1440, 1479, 1481, 1492, 1496, 1545,
       1554, 1566, 1567, 1571, 1615, 1675, 1715, 1741, 1754, 1764, 1811,
       1831, 1838, 1842, 1860, 1884, 1890, 1909, 1984, 1987, 1993, 2017,
       2018, 2050, 2076, 2097, 2143, 2154, 2168, 2170, 2173, 2194, 2195,
       2203, 2206, 2224, 2230, 2247, 2255, 2282, 2308, 2314, 2319, 2322,
       2325, 2328, 2344, 2405, 2408, 2434, 2470, 2478, 2506, 2508, 2540,
       2562, 2579, 2588, 2628, 2632, 2636, 2649, 2657, 2676, 2712, 2755,
       2779, 2793, 2815, 2825, 2827, 2831, 2838, 2871, 2879, 2904, 2906,
       2912, 2925, 2936, 2940, 2941, 2945, 3002, 3004, 3008, 3037, 3041,
       3058, 3090, 3095, 3123, 3126, 3162, 3163, 3182, 3201, 3206, 3220,
       3225, 3241, 3244, 3252, 3269, 3272, 3306, 3338, 3339, 3365, 3366,
       3389, 3427, 3430, 3439, 3481, 3502, 3529, 35

In [181]:
id_list

NameError: name 'id_list' is not defined

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [ ]:
}